In [ ]:
import os
from glob import glob
import nibabel as nib
from numpy import median, rint, exp
from os.path import join as opj

from nipype.interfaces import freesurfer as fs
from nipype.interfaces import spm
from nipype.interfaces import fsl
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import FreeSurferSource, SelectFiles, DataSink
from nipype.interfaces.slicer.registration import brainsresample
from nipype.algorithms.misc import Gunzip
from nipype.pipeline.engine import Workflow, Node, MapNode

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

#freesurfer setup
fs_dir = subjects_dir + '/freesurfer'
fs.FSCommand.set_default_subjects_dir(fs_dir)

#other study-specific variables
project_home = '/Users/catcamacho/Dropbox/Projects/TH_NAR_ASL/proc/dev'
subjects_dir = project_home
subjects_list = ['ko']
raw_dir = project_home + '/raw'
output_dir = project_home + '/proc'
template = project_home + '/template/avg152T1.nii'

#Population specific variables for ASL
nex_asl = 1 #number of excitations from the 3D ASL scan parameters
inversion_efficiency = 0.8 #from GE
background_supp_eff = 0.75 #from GE
efficiency = inversion_efficiency * background_supp_eff 
T1_blood = 1.6 #T1 of blood in seconds(1.6s at 3T and 1.4s at 1.5T)
sat_time = 2 #in seconds, from GE
partition_coeff = 0.9 #whole brain average in ml/g
scaling_factor = 32 #scaling factor, can be taken from PW dicom header at position 0043,107f
postlabel_delay = 2.025 #1.525 #post label delay in seconds
labeling_time = 1.450 #labeling time in seconds
T1_tissue = 1.2 #estimated T1 of grey matter in seconds

In [ ]:
## File handling nodes

# Select subjects
infosource = Node(IdentityInterface(fields=['subjid']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

# SelectFiles
templates = {'cbf': raw_dir + '/{subjid}/pw.nii'}
selectfiles = Node(SelectFiles(templates), name='selectfiles')

# FreeSurferSource - Data grabber specific for FreeSurfer data
fssource = Node(FreeSurferSource(subjects_dir=fs_dir),
                run_without_submitting=True,
                name='fssource')
# Datasink
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir

In [ ]:
## Custom functions

# quantify CBF from PW volume
def quantify_cbf(pw_volume,pd_volume,sat_time,T1_tissue,postlabel_delay,T1_blood,labeling_time,efficiency,nex_asl,scaling_factor,partition_coeff):
    # set variables
    pw_data = pw_volume.get_data()
    pd_data = pd_volume.get_data()
    conversion = 6000 #to convert values from mL/g/s to mL/100g/min
    
    cbf_numerator = (1-exp(-1*sat_time/T1_tissue))*exp(postlabel_delay/T1_blood)
    cbf_denominator = 2*T1_blood*(1-exp(-1*labeling_time/T1_blood))*efficiency*nex_asl
    cbf_data = conversion*partition_coeff*(cbf_numerator/cbf_denominator)*(pw_data/(scaling_factor*pd_data))
    
    cbf_vol = nib.Nifti1Image(cbf_data, pw_nifti1.affine)
    nib.save(cbf_vol,raw_dir + '/{subjid}/cbf.nii')
    
quant_cbf = Node(name='quant_cbf',
                interface=Function(input_names=[],
                                  output_names=['out_file'],
                                  function=quantify_cbf))

In [ ]:
## File Processing nodes

# reorient data for consistency
reorient = Node(fsl.utils.Reorient2Std(output_type='NIFTI'),
                name='reorient')

# BBRegister - coregister a volume to the Freesurfer anatomical
bbregister = Node(fs.BBRegister(init='header',
                                contrast_type='t2',
                                out_fsl_file=True),
                  name='bbregister')

# convert files to nifti
mri_convert = Node(fs.MRIConvert(out_type='nii',
                                out_orientation='RAS'), 
                   name='mri_convert')

# Volume Transformation - transform the brainmask into functional space
applyVolTrans = Node(fs.ApplyVolTransform(inverse=True),
                     name='applyVolTrans')

# Binarize -  binarize and dilate image to create a brainmask
binarize = Node(fs.Binarize(min=0.5,
                         dilate=1,
                         out_type='nii'),
                name='binarize')

# Mask brain in cbf volume
applyMask = Node(fsl.maths.ApplyMask(output_type='NIFTI'), 
                        name='applyMask')

In [ ]:
# Create a preprocessing for anat + CBF processing
cbfprocflow = Workflow(name='cbfprocflow')

# Connect all components of the preprocessing workflow
cbfprocflow.connect([(infosource, selectfiles, [('subjid', 'subjid')]),
                     (infosource, fssource, [('subjid','subject_id')]),
                     (infosource, bbregister, [('subjid','subject_id')]),
                     (selectfiles, bbregister, [('cbf', 'source_file')]),
                     (fssource, mri_convert, [('brainmask', 'in_file')]),
                     (infosource, applyVolTrans, [('subjid','subject')]),
                     (mri_convert, applyVolTrans, [('out_file', 'target_file')]),
                     (bbregister, applyVolTrans, [('out_reg_file', 'reg_file')]),
                     (selectfiles, applyVolTrans, [('cbf', 'source_file')]),
                     (applyVolTrans, datasink, [('transformed_file', 'reg_anat')]),
                     (applyVolTrans, binarize, [('transformed_file', 'in_file')]),
                     (binarize, datasink, [('binary_file', 'binary_anat')]),
                     (binarize, applyMask, [('binary_file','mask_file')]),
                     (selectfiles, applyMask, [('cbf','in_file')]),
                     (applyMask, datasink, [('out_file','masked_cbf')])
                     ])
cbfprocflow.base_dir = opj(subjects_dir)
cbfprocflow.write_graph(graph2use='flat')
cbfprocflow.run('MultiProc', plugin_args={'n_procs': 2})